<p align="center"
    <a href="https://colab.research.google.com/github/ContextLab/davos/blob/main/tests/test-davos-colab.ipynb" target="_parent">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab">
    </a>
</p>

In [1]:
GITHUB_USERNAME = "$GITHUB_USERNAME"
GITHUB_REF = "$GITHUB_REF"

In [2]:
import ast
import contextlib
import functools
import html
import os
import signal
import subprocess
import sys
import traceback
import types
from pathlib import Path

import google
import IPython
import pkg_resources
from IPython.core.ultratb import FormattedTB
from IPython.display import display_html
from IPython.utils.io import capture_output as capture_ipython_display

In [3]:
def install_davos(source='github', ref=None, fork=None):
    """
    Install a particular version or revision of davos from 
    the specified remote source

    Parameters
    ----------
    source : {'github', 'pip', 'pypi', 'testpypi', 'conda'}, 
             default: 'github'
        The remote source from which to install davos. GitHub 
        is generally used for CI tests, pip/pypi for full 
        releases, and testpypi for test releases
    
    ref : str, optional
        The version or revision of davos to install. If 
        source is 'github', this can be a branch, commit hash, 
        or tag. Otherwise, this may be a valid version string 
        to install from the given source. Defaults to most 
        recent revision on the default branch of the specified 
        fork (GitHub) or the latest release version (others).

    fork : str, optional
        Optionally, the fork (GitHub username) to install from. 
        Defaults to the base repository (ContextLab). If source 
        is not 'github', this has no effect.
    
    """
    source = source.lower()
    if source == 'github':
        if fork is None:
            fork = 'ContextLab'
        name = f'git+https://github.com/{fork}/davos.git'
        if ref is not None:
            name += f'@{ref}'
        name += '#egg=davos'
    elif source in ('pip', 'pypi', 'testpypi'):
        name = 'davos'
        if source == 'testpypi':
            name = '--index-url https://test.pypi.org/simple/ ' + name
        if ref is not None:
            name += f'=={ref}'
    elif source == 'conda':
        raise NotImplementedError(
            "conda installation is not supported in Colaboratory"
        )
    else:
        raise ValueError(f"Invalid source '{source}'")
    
    return_code = IPython.core.interactiveshell.system(f'pip install {name}')
    if return_code != 0:
        raise DavosTestingError(
            f"Failed to install 'davos'. Ran command:\n\t`pip install {name}`"
        )

In [4]:
install_davos(source='github', ref=GITHUB_REF, fork=GITHUB_USERNAME)
import davos

  Cloning https://github.com/paxtonfitzpatrick/davos.git (to revision tests) to /tmp/pip-install-epfln8gj/davos
  Running command git clone -q https://github.com/paxtonfitzpatrick/davos.git /tmp/pip-install-epfln8gj/davos
  Running command git checkout -b tests --track origin/tests
  Switched to a new branch 'tests'
  Branch 'tests' set up to track remote branch 'tests' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for davos: filename=davos-0.0.1-cp37-none-any.whl size=50199 sha256=fe46bb2eba7f3ba48becfd6638aeb9c8759a84144305f0e6893a694f048be44a
  Stored in directory: /tmp/pip-ephem-wheel-cache-9vf_t3x2/wheels/0e/6a/2a/906cde07aa724e6b4f5a6a744f7b05a2a4cf4462549a5b401f
Successfully built davos


In [5]:
IPYTHON_SHELL = get_ipython()

# **Test helpers**

In [6]:
class DavosTestingError:
    """Base class for Davos testing-related errors"""
    pass


class DavosAssertionError(DavosTestingError, AssertionError):
    """Subclasses AssertionError for pytest-specific handling"""
    pass


class TestingEnvironmentError(DavosAssertionError, OSError):
    """Raised due to issues with the testing environment"""
    pass


class NonColabEnvironmentError(TestingEnvironmentError):
    """Raised if this notebook is run outside of Colaboratory"""
    def __init__(self, *args):
        msg = "Tests in this notebook are meant to be run in a Colaboratory environment"
        super().__init__(msg, *args)


class TestTimeoutError(DavosAssertionError):
    """Raised if a test exceeds its timeout limit"""
    pass

In [7]:
def _expected_onion_parser_output(args_str, **installer_kwargs):
    _installer_kwargs = {'editable': False, 'spec': args_str}
    _installer_kwargs.update(installer_kwargs)
    return '"pip"', f'"""{args_str}"""', _installer_kwargs

In [8]:
def _expected_parser_output(name, as_=None, args_str=None, **installer_kwargs):
    if as_ is not None:
        as_ = f'"{as_}"'
    expected = f'smuggle(name="{name}", as_={as_}'
    if args_str is not None or any(installer_kwargs):
        installer, args_str, inst_kwargs = _expected_onion_parser_output(args_str, **installer_kwargs)
        expected += f', installer={installer}, args_str={args_str}, installer_kwargs={inst_kwargs}'
    return expected + ')'

In [9]:
def _is_imported(package):
    if package in sys.modules:
        return True
    elif any(pkg.startswith(f'{package}.') for pkg in sys.modules):
        return True
    return False

In [10]:
def _is_installed(package):
    try:
        dist = pkg_resources.get_distribution(package)
    except pkg_resources.DistributionNotFound:
        return False
    else:
        return True

In [11]:
def _matches_expected_output(expected, result):
    all_match = True
    expected_chunks = expected.split(';')
    result_chunks = result.split(';')
    assert len(expected_chunks) == len(result_chunks)
    for expected_chunk, result_chunk in zip(expected_chunks, result_chunks):
        if 'installer_kwargs=' in expected_chunk and 'installer_kwargs' in result_chunk:
            exp_main, exp_kwargs = expected_chunk.strip().split('installer_kwargs=')
            res_main, res_kwargs = result_chunk.strip().split('installer_kwargs=')
            # remove ')'
            exp_kwargs = ast.literal_eval(exp_kwargs[:-1])
            res_kwargs = ast.literal_eval(res_kwargs[:-1])
            all_match = all_match and exp_main == res_main and exp_kwargs == res_kwargs
        else:
            all_match = all_match and expected_chunk == result_chunk
    return all_match

In [12]:
def _parse_line(line):
    return davos.implementations.full_parser(line)

In [13]:
def _parse_onion(onion):
    return davos.core.core.Onion.parse_onion(onion)

In [14]:
def format_traceback(err):
    tb_formatter = FormattedTB('Context', 'NoColor')
    structured_tb = tb_formatter.structured_traceback(type(err), err, 
                                                      err.__traceback__)
    tb_string = tb_formatter.stb2text(structured_tb)
    return html.escape(tb_string)

In [15]:
def mark_timeout(timeout=120):
    # stand-in for @pytest.mark.timeout (from pytest-timeout plugin)
    def decorator(func):
        def handler(signum, frame):
            raise TestTimeoutError(
                f"'{func.__name__}' timed out after {timeout} seconds"
            )
        
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, handler)
            signal.alarm(timeout)
            try:
                return func(*args, **kwargs)
            finally:
                signal.alarm(0)

        return wrapper
    return decorator

In [16]:
def run_tests():
    tests = []
    for name, obj in globals().items():
        if name.startswith('test_') and isinstance(obj, types.FunctionType):
            tests.append((name, obj))

    print(f"collected {len(tests)} items\n")
    for test_name, test_func in tests:
        try:
            with open(os.devnull, 'w') as devnull, contextlib.redirect_stdout(devnull):
                test_func()
        except Exception as e:
            tb = format_traceback(e)
            status = f'FAILED\n{"-"*40}\n{tb}\n{"-"*40}'
        else:
            status = 'PASSED'
        
        html = f"<div id='{test_name}_result' style=\"white-space:pre\">{test_name}\t{status}</div>"
        display_html(html, raw=True)

# **Notebook environment tests**

In [17]:
def test_ipython_shell_is_colab_shell():
    try:
        assert isinstance(IPYTHON_SHELL, google.colab._shell.Shell)
    except AttributeError as e:
        google_version = pkg_resources.get_distribution('google').version
        raise TestingEnvironmentError(
            "Qualified name for Colab interactive shell class has changed "
            "(google module may have been recently updated).\n\tShell type:\t"
            f"{type(IPYTHON_SHELL)}\n\tgoogle version:\t{google_version}"
        ) from e

In [18]:
def test_ipython_version():
    ipy_version = IPython.__version__
    try:
        assert ipy_version == '5.5.0'
    except AssertionError as e:
        raise TestingEnvironmentError(
            f"Expected IPython==5.5.0, found IPython=={ipy_version}. Colab "
            "package environment may have been recently updated"
        ) from e

# **Initialization tests**

In [19]:
def test_smuggle_function_in_namespace():
    assert 'smuggle' in IPYTHON_SHELL.user_ns

In [20]:
def test_input_transformer_registered():
    splitter_transforms = IPYTHON_SHELL.input_splitter.python_line_transforms
    line_transforms = IPYTHON_SHELL.input_transformer_manager.python_line_transforms
    # transform be registered once (and only once) in both places
    assert len(splitter_transforms) == 1
    assert len(line_transforms) == 1
    smuggler_splitter_transform = splitter_transforms[0]
    smuggler_line_transform = line_transforms[0]
    # both should be StatelessInputTransformer instances
    assert isinstance(smuggler_splitter_transform, 
                      IPython.core.inputtransformer.StatelessInputTransformer)
    assert isinstance(smuggler_line_transform, 
                      IPython.core.inputtransformer.StatelessInputTransformer)
    # both objects' .func attr should be "full" parser function, which for 
    # IPython<7.0 should simply be the parse_line function 
    assert (smuggler_splitter_transform.func 
            is smuggler_line_transform.func 
            is davos.implementations.full_parser
            is davos.core.core.parse_line)

In [21]:
def test_DavosConfig_object_initialization():
    config = davos.config
    # config object should be a singleton
    assert davos.DavosConfig() is config
    # should recognize Colab notebook environment
    assert config.environment == 'Colaboratory'
    # global IPython shell instance and its original showsyntaxerror 
    # method should be stored as expected
    assert (config.ipython_shell 
            is IPYTHON_SHELL 
            is config._ipy_showsyntaxerror_orig.__self__)
    # conda executable should be unavailable...
    assert config.conda_avail is False
    # ...so name of current conda environment should be None...
    assert config.conda_env is None
    # ...and env name - env dir path mapping should also be None
    assert config.conda_envs_dirs is None
    # dict of previously smuggled packages should initially be empty
    assert config.smuggled == {}
    # davos parser should initially be active
    assert config.active is davos.is_active() is True
    # automatic restart & run all above behavior should be unavailable in colab
    assert config.allow_rerun is False
    # install confirmation should not be required by default
    assert config.confirm_install is False
    # noninteractive mode should be disabled by default
    assert config.noninteractive is False
    # stdout should not be suppressed by default
    assert config.suppress_stdout is False

# **Main tests**

## Unit tests

### shell command runner tests

In [22]:
def test_run_shell_command_simple():
    stdout = davos.core.core.run_shell_command('whoami', live_stdout=False)
    assert stdout == 'root\r\n'

In [23]:
def test_run_shell_command_multiword():
    quote = ("Strictly speaking, I didn't do the theiving. That would be the "
             "pirates. I just moved what they stole from one place to another")
    stdout = davos.core.core.run_shell_command(f'echo "{quote}"', 
                                               live_stdout=False)
    assert stdout == quote + '\r\n'

In [24]:
def test_run_shell_command_failure():
    try:
        davos.core.core.run_shell_command('blahblahblah', live_stdout=True)
    except subprocess.CalledProcessError as e:
        assert e.returncode == 127
        assert e.output == '/bin/sh: 1: blahblahblah: not found\r\n', e.output.getvalue()

### smuggle command parser tests

In [25]:
def test_parser_ignores_line_no_smuggle():
    line = "def foo(bar, baz=qux):"
    assert _parse_line(line) == line

In [26]:
def test_parser_ignores_line_decoy_smuggle():
    """a line that has "smuggle" in it, but should be ignored"""
    line = "def smuggle_something(foo):"
    assert _parse_line(line) == line

In [27]:
def test_parser_ignores_commented_line():
    """parser should ignore commented-out smuggle statements"""
    line = "# smuggle foo as bar"
    assert _parse_line(line) == line

In [28]:
def test_parser_handles_basic_line():
    """simplest use case"""
    line = "smuggle foo"
    expected = _expected_parser_output('foo')
    assert _matches_expected_output(expected, _parse_line(line))

In [29]:
def test_parser_handles_basic_line_alias():
    """simplest use case, plus alias"""
    line = "smuggle foo as bar"
    expected = _expected_parser_output('foo', as_='bar')
    assert _matches_expected_output(expected, _parse_line(line))

In [30]:
def test_parser_handles_basic_line_onion():
    """simplest use case, with onion comment"""
    line = "smuggle foo as bar    # pip: foo==0.0.1"
    expected = _expected_parser_output('foo', as_='bar', args_str='foo==0.0.1')
    assert _matches_expected_output(expected, _parse_line(line))

In [31]:
def test_parser_ignores_non_onion_comment():
    """looks similar to the preivous one, but the inline comment is NOT an onion spec"""
    line = "smuggle foo as bar    # omg foo is my FAVORITE package"
    expected = _expected_parser_output('foo', as_='bar') + "    # omg foo is my FAVORITE package"
    assert _matches_expected_output(expected, _parse_line(line))

In [32]:
def test_parser_ignores_imposter_onion_comment():
    """VERY close to an onion comment, but doesn't start with "pip:" (note colon)"""
    line = "smuggle foo as bar    # pip is a [redacted] package manager than conda"
    expected = _expected_parser_output('foo', as_='bar') + "    # pip is a [redacted] package manager than conda"
    assert _matches_expected_output(expected, _parse_line(line))

In [33]:
def test_parser_handles_smuggle_qualname():
    line = "smuggle foo.bar as baz"
    expected = _expected_parser_output('foo.bar', as_='baz')
    assert _matches_expected_output(expected, _parse_line(line))

In [34]:
def test_parser_handles_indented_line():
    """
    needs to handle, e.g.
        ```
        def foo():
            smuggle numpy as np
        ```
    Note: unless `davos.deactivate() is run, the above line will 
    actually be parsed (though it will have no effect)
    """
    line = "    smuggle foo as bar"
    expected = "    " + _expected_parser_output('foo', as_='bar')
    assert _matches_expected_output(expected, _parse_line(line))

In [35]:
def test_parser_handles_different_install_name():
    """
    sometimes, name used for pip-installation is different 
    from name used to import
    """
    line = "smuggle foo as bar    # pip: foo-package==0.0.1"
    expected = _expected_parser_output('foo', as_='bar', 
                                       args_str='foo-package==0.0.1')
    assert _matches_expected_output(expected, _parse_line(line))

In [36]:
def test_parser_handles_multiple_packages():
    line = "smuggle foo as bar, baz as qux, spam, ham, eggs"
    expected = _expected_parser_output('foo', as_='bar')
    expected += f"; {_expected_parser_output('baz', as_='qux')}"
    expected += f"; {_expected_parser_output('spam')}"
    expected += f"; {_expected_parser_output('ham')}"
    expected += f"; {_expected_parser_output('eggs')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [37]:
def test_parser_handles_multiple_packages_onion():
    """onion info should be passed to smuggle function for FIRST package"""
    line = "smuggle foo as bar, baz as qux, spam, ham, eggs    # pip: foo==0.0.1"
    expected = _expected_parser_output('foo', as_='bar', args_str='foo==0.0.1')
    expected += f"; {_expected_parser_output('baz', as_='qux')}"
    expected += f"; {_expected_parser_output('spam')}"
    expected += f"; {_expected_parser_output('ham')}"
    expected += f"; {_expected_parser_output('eggs')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [38]:
def test_parser_handles_backslash():
    line = """smuggle foo as bar, \
                      baz as qux, \
                      spam, ham, \
                      eggs"""
    expected = _expected_parser_output('foo', as_='bar')
    expected += f"; {_expected_parser_output('baz', as_='qux')}"
    expected += f"; {_expected_parser_output('spam')}"
    expected += f"; {_expected_parser_output('ham')}"
    expected += f"; {_expected_parser_output('eggs')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [39]:
def test_parser_handles_backslash_onion():
    """again, onion info should be passed to smuggle function for FIRST package"""
    line = """smuggle foo as bar, \
                      baz as qux, \
                      spam, ham, \
                      eggs    # pip: foo==0.0.1"""
    expected = _expected_parser_output('foo', as_='bar', args_str='foo==0.0.1')
    expected += f"; {_expected_parser_output('baz', as_='qux')}"
    expected += f"; {_expected_parser_output('spam')}"
    expected += f"; {_expected_parser_output('ham')}"
    expected += f"; {_expected_parser_output('eggs')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [40]:
def test_parser_handles_inconsistent_whitespace():
    """should handle weird amounts of whitespace that are *technically* valid"""
    line = """smuggle               foo     as    bar    \
,baz as qux  , \
          spam  .  ham    as    eggs                #   pip   :       foo==0.0.1    """
    expected = _expected_parser_output('foo', as_='bar', args_str='foo==0.0.1')
    expected += f"; {_expected_parser_output('baz', as_='qux')}"
    expected += f"; {_expected_parser_output('spam.ham', as_='eggs')}"
    # real parser adds back leading & trailing characters (including whitespace)
    expected += "    "
    assert _matches_expected_output(expected, _parse_line(line))

In [41]:
def test_parser_handles_smuggle_from():
    """second possible broad syntax class"""
    line = "from foo smuggle bar"
    expected = _expected_parser_output('foo.bar', as_='bar')
    assert _matches_expected_output(expected, _parse_line(line))

In [42]:
def test_parser_handles_smuggle_from_onion():
    """second possible broad syntax class"""
    line = "from foo smuggle bar    # pip: foo==0.0.1"
    expected = _expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    assert _matches_expected_output(expected, _parse_line(line))

In [43]:
def test_parser_handles_smuggle_from_multi():
    line = "from foo smuggle bar, baz as spam, qux"
    expected = _expected_parser_output('foo.bar', as_='bar')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [44]:
def test_parser_handles_smuggle_from_multi_onion():
    """onion info should be passed to FIRST smuggle function"""
    line = "from foo smuggle bar, baz as spam, qux    # pip: foo==0.0.1"
    expected = _expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [45]:
def test_parser_handles_smuggle_from_backslash():
    """onion info should be passed to FIRST smuggle function"""
    line = """from foo smuggle bar, \
                               baz as spam, \
                               qux    # pip: foo==0.0.1"""
    expected = _expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [46]:
def test_parser_handles_smuggle_from_parentheses():
    line = """from foo smuggle (bar, baz as spam, qux,)"""
    # also tests trailing comma inside parentheses, which is valid
    expected = _expected_parser_output('foo.bar', as_='bar')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [47]:
def test_parser_handles_smuggle_from_parentheses_multiline_1():
    line = """from foo smuggle (bar, 
                                baz as spam, 
                                qux)"""
    expected = _expected_parser_output('foo.bar', as_='bar')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [48]:
def test_parser_handles_smuggle_from_parentheses_multiline_1_onion_1():
    line = """from foo smuggle (bar,    # pip: foo==0.0.1
                                baz as spam, 
                                qux)"""
    expected = _expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [49]:
def test_parser_handles_smuggle_from_parentheses_multiline_1_onion_2():
    line = """from foo smuggle (bar, 
                                baz as spam, 
                                qux)    # pip: foo==0.0.1"""
    expected = _expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [50]:
def test_parser_handles_smuggle_from_parentheses_multiline_2():
    line = """from foo smuggle (
                  bar, 
                  baz as spam, 
                  qux
              )"""
    expected = _expected_parser_output('foo.bar', as_='bar')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [51]:
def test_parser_handles_smuggle_from_parentheses_multiline_2_onion_1():
    line = """from foo smuggle (    # pip: foo==0.0.1
                  bar, 
                  baz as spam, 
                  qux
              )"""
    expected = _expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [52]:
def test_parser_handles_smuggle_from_parentheses_multiline_2_onion_2():
    line = """from foo smuggle (
                  bar, 
                  baz as spam, 
                  qux
              )    # pip: foo==0.0.1"""
    expected = _expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    assert _matches_expected_output(expected, _parse_line(line))

In [53]:
def test_parser_handles_smuggle_from_parentheses_multiline_2_onion_2_comments():
    line = """from foo smuggle (    # unrelated comment on first line
                  bar,    # unrelated comment on package name line
                  baz as spam, 
                  # unrelated comment on its own line
                  qux
              )    # pip: foo==0.0.1    # unrelated comment after onion"""
    expected = _expected_parser_output('foo.bar', as_='bar', args_str='foo==0.0.1')
    expected += f"; {_expected_parser_output('foo.baz', as_='spam')}"
    expected += f"; {_expected_parser_output('foo.qux', as_='qux')}"
    expected += "    # unrelated comment after onion"
    assert _matches_expected_output(expected, _parse_line(line))

In [54]:
def test_parser_handles_smuggle_semicolons():
    """also combines multiple elements from prior tests"""
    line = """smuggle foo; smuggle bar as baz; \
              from spam smuggle eggs; \
              from qux smuggle quux as corge    # pip: qux-package==0.0.1"""
    expected = _expected_parser_output('foo')
    expected += f"; {_expected_parser_output('bar', as_='baz')}"
    expected += f"; {_expected_parser_output('spam.eggs', as_='eggs')}"
    expected += f"; {_expected_parser_output('qux.quux', as_='corge', args_str='qux-package==0.0.1')}"
    assert _matches_expected_output(expected, _parse_line(line))

### onion comment parser tests

In [55]:
def test_onion_parser_simple():
    onion = '# pip: foo==0.0.1'
    expected = _expected_onion_parser_output('foo==0.0.1')
    assert _parse_onion(onion) == expected

In [56]:
def test_onion_parser_whitespace():
    onion = '#             pip:            foo==0.0.1'
    expected = _expected_onion_parser_output('foo==0.0.1')
    assert _parse_onion(onion) == expected

In [57]:
def test_onion_parser_github():
    onion = '# pip: git+https://github.com/foo/bar.git@branch-name#egg=foo&subdirectory=baz'
    expected = _expected_onion_parser_output('git+https://github.com/foo/bar.git@branch-name#egg=foo&subdirectory=baz')
    assert _parse_onion(onion) == expected

In [58]:
def test_onion_parser_editable():
    onion = '# pip: --editable git+https://github.com/foo/bar.git'
    expected = _expected_onion_parser_output('--editable git+https://github.com/foo/bar.git', 
                                             editable=True, 
                                             spec='git+https://github.com/foo/bar.git')
    assert _parse_onion(onion) == expected

In [59]:
def test_onion_parser_joined_short_args():
    onion = '# pip: -Ive git+https://github.com/foo/bar.git'
    expected = _expected_onion_parser_output('-Ive git+https://github.com/foo/bar.git', 
                                             editable=True, 
                                             ignore_installed=True, 
                                             verbosity=1,
                                             spec='git+https://github.com/foo/bar.git')
    assert _parse_onion(onion) == expected

In [60]:
def test_onion_parser_fails_mutually_exclusive_args():
    onion = '# pip: --use-pep517 --no-use-pep517 foo'
    try:
        _parse_onion(onion)    # SHOULD FAIL
    except davos.core.exceptions.OnionArgumentError as e:
        # has both 'msg' and 'message' attrs because it inherits from both 
        # SyntaxError and argparse.ArgumentError
        assert e.msg == e.message == "not allowed with argument --use-pep517"
        assert e.argument_name == "--no-use-pep517"
        assert str(e) == "argument --no-use-pep517: not allowed with argument --use-pep517"
    else:
        raise DavosAssertionError("test should've raised 'davos.core.exceptions.OnionArgumentError'")

In [61]:
def test_onion_parser_fails_editable_after_spec():
    onion = '# pip: git+https://github.com/foo/bar.git#egg=foo --editable'
    try:
        _parse_onion(onion)    # SHOULD FAIL
    except davos.core.exceptions.OnionArgumentError as e:
        # has both 'msg' and 'message' attrs because it inherits from both 
        # SyntaxError and argparse.ArgumentError
        assert e.msg == e.message == "expected one argument"
        assert e.argument_name == "-e/--editable"
        assert str(e) == "argument -e/--editable: expected one argument"
    else:
        raise DavosAssertionError("test should've raised 'davos.core.exceptions.OnionArgumentError'")

In [62]:
def test_onion_parser_fails_arg_requires_value():
    onion = '# pip: foo==0.0.1 --platform '
    try:
        _parse_onion(onion)    # SHOULD FAIL
    except davos.core.exceptions.OnionArgumentError as e:
        # has both 'msg' and 'message' attrs because it inherits from both 
        # SyntaxError and argparse.ArgumentError
        assert e.msg == e.message == "expected one argument"
        assert e.argument_name == "--platform"
        assert str(e) == "argument --platform: expected one argument"
    else:
        raise DavosAssertionError("test should've raised 'davos.core.exceptions.OnionArgumentError'")

### miscellaneous tests

In [63]:
def test_does_not_register_multiple_transformers():
    splitter_xforms = IPYTHON_SHELL.input_splitter.python_line_transforms
    manager_xforms = IPYTHON_SHELL.input_transformer_manager.python_line_transforms
    initial_n_splitter_xforms = len(splitter_xforms)
    initial_n_manager_xforms = len(manager_xforms)
    # record whether parser is active before running test to restore 
    # initial state afterward
    start_active = davos.is_active()
    if not start_active:
        # ensure these are the number of registered transforms 
        # *including the davos parser*
        initial_n_splitter_xforms += 1
        initial_n_manager_xforms += 1

    try:
        for _ in range(5):
            davos.activate()
            assert len(splitter_xforms) == initial_n_splitter_xforms
            assert len(manager_xforms) == initial_n_manager_xforms
    finally:
        if not start_active:
            davos.deactivate()

In [64]:
def test_deactivate_reactivate():
    start_active = davos.is_active()
    try:
        # start with the parser active
        if not start_active:
            davos.activate()
        
        assert davos.is_active()
        davos.deactivate()
        assert not davos.is_active()
        davos.activate()
        assert davos.is_active()
    finally:
        # return to state before function
        if not start_active:
            davos.deactivate()

## Integration tests

In [65]:
@mark_timeout(30)
def test_smuggle_pypi_new():
    assert not _is_installed('ppca')
    smuggle ppca    # pip: ppca>=0.0.4
    assert isinstance(ppca, types.ModuleType)
    assert hasattr(ppca, 'PPCA')

In [66]:
@mark_timeout(30)
def test_smuggle_from_pypi_new():
    assert not _is_installed('umap-learn')
    from umap smuggle UMAP    # pip: umap-learn==0.4.6
    assert hasattr(UMAP, 'fit')
    assert _is_installed('umap-learn==0.4.6')

In [67]:
@mark_timeout(30)
def test_smuggle_previously_installed():
    assert _is_installed('fastdtw==0.3.4')
    smuggle fastdtw    # pip: fastdtw==0.3.2
    assert _is_installed('fastdtw==0.3.2')

In [68]:
@mark_timeout(30)
def test_smuggle_previously_imported():
    import tqdm
    assert tqdm.__version__ == '4.41.1'
    smuggle tqdm    # pip: tqdm==4.45.0
    assert tqdm.__version__ == '4.45.0'

In [69]:
@mark_timeout(30)
def test_smuggle_github_ref():
    assert not _is_installed('hypertools')
    smuggle hypertools as hyp    # pip: git+https://github.com/ContextLab/hypertools.git@36c12fd#egg=hypertools
    assert isinstance(hyp, types.ModuleType)
    assert hyp is sys.modules['hypertools']
    assert hasattr(hyp, 'DataGeometry')
    assert hyp.__version__ == '0.6.3'
    assert _is_installed('hypertools==0.6.3')

In [70]:
@mark_timeout(30)
def test_smuggle_github_editable():
    assert not _is_installed('quail')
    smuggle quail    # pip: -e git+https://github.com/ContextLab/quail.git@v0.2.0#egg=quail --src /content/gh_clones
    assert '/content/gh_clones/quail' in sys.path
    assert isinstance(quail, types.ModuleType)
    assert hasattr(quail, 'Egg')
    assert _is_installed('quail==0.2.0')
    assert Path('/content/gh_clones/quail').is_dir()

In [71]:
@mark_timeout(300)
def test_smuggle_github_subdirectory():
    assert not _is_installed('sherlock_helpers')
    with capture_ipython_display():
        # displays an IPython object on import, which 
        # contextlib.redirect_stdout doesn't catch
        smuggle sherlock_helpers    # pip: git+https://github.com/ContextLab/sherlock-topic-model-paper.git@v1.0#subdirectory=code/sherlock_helpers
    assert isinstance(sherlock_helpers, types.ModuleType)
    assert hasattr(sherlock_helpers, 'github_url')
    assert sherlock_helpers.__version__ == '0.0.1'
    assert _is_installed('sherlock_helpers==0.0.1')

# **Run tests**

In [72]:
run_tests()

collected 55 items



test_ipython_shell_is_colab_shell	PASSED

test_ipython_version	PASSED

test_smuggle_function_in_namespace	PASSED

test_input_transformer_registered	PASSED

...

test_smuggle_previously_installed	PASSED

test_smuggle_previously_imported	PASSED

test_smuggle_github_ref	PASSED

test_smuggle_github_editable	PASSED

test_smuggle_github_subdirectory	PASSED